In [0]:
# default_exp data.exploratory.se.metrics

# Exploration of your data

> This module comprises some of the statistical and inference techniques to describe the inner properties of software data. The submodules might include:
>
> - Descriptive statistics
> - Software Metrics
> - Information Theory
> - Learning Principels Detection (Occams' Razor, Biased data, and Data Snooping)
> - Inference: Probabilistic and Causal

### Specifically in this module

> - Cyclomatic complexity (CYCLO)
> - Coupling Between Objects (CBO)
> - Lack of Cohesion of Methods 5 (LCOM5)
> - Top/Least 20 Occurring Tokens
> - Percentage of redundancy (just Java)
> - How many Type I and Type II clones are in Java datasets?

In [0]:
# #hide
# from nbdev.showdoc import *

In [3]:
!pip install sentencepiece

In [0]:
# export
# Imports
import pandas as pd
import sentencepiece as sp
from numpy import mean, std
from statistics import median
from scipy.stats import sem, t

# TODO: Remove when mongo call is implemented
import os

In [0]:
# TODO: Replace with actual mongo call
def simulate_getting_dataframes_from_mongo():
    corpus_data = {'file_name': [], 'data_type': [], 'contents': []}
    path = "./source_code"
    for file in os.listdir(path):
        corpus_data['file_name'].append(file)
        corpus_data['data_type'].append('src')
        with open (os.path.join(path, file), "r") as f:
            corpus_data['contents'].append(f.read())
    path = "./tests"
    for file in os.listdir(path):
        corpus_data['file_name'].append(file)
        corpus_data['data_type'].append('test')
        with open (os.path.join(path, file), "r") as f:
            corpus_data['contents'].append(f.read())
    corpus_df = pd.DataFrame(data = corpus_data)
    return corpus_df

In [0]:
# export
# TODO: Change based on format of input data from Mongo
def df_to_txt_file(df, output, cols):
    """Converts a dataframe into a text file that SentencePiece can use to train a BPE model"""
    if cols is None: cols = list(df.columns)
    merged_df = pd.concat([df[col] for col in cols])
    
    with open(output + '_text.txt', 'w') as f:
        f.write('\n'.join(list(merged_df)))
    return output + '_text.txt'

In [0]:
# export
# TODO: Change based on format of input data from Mongo
def gen_sp_model(df, output, model_name, cols=None):
    """Trains a SentencePiece BPE model from a pandas dataframe"""
    fname = df_to_txt_file(df, output, cols)
    sp.SentencePieceTrainer.train(f'--input={fname} --model_prefix={output + model_name} --hard_vocab_limit=false --model_type=bpe')
    return output + model_name

In [0]:
# export
def add_length_col(df, col, length_col_name='length'):
    """Adds a length column with the length of entries in the specified column of a dataframe"""
    df[length_col_name] = df[col].astype(str).apply(len)
    return df

In [0]:
# export
def stat_metrics(df, col, data_types=None, conf = 0.95, sig_figs = 4):
    """Computes statistical metrics about the entries in a dataframe column"""
    if data_types != None: 
        df = df[df['data_type'].isin(data_types)]
        print(f"~~Printing metrics for {data_types} in column [{col}]~~")
    else:
        print(f"~~Printing metrics for all data in column [{col}]~~")

    print("Min =", round(df[col].min(), sig_figs))
    print("Max =", round(df[col].max(), sig_figs))
    print("Average =", round(df[col].mean(), sig_figs))
    print("Median =", round(median(df[col]), sig_figs))
    print("Standard Deviation =", round(std(df[col]), sig_figs))
    
    n = len(df[col])
    m = mean(df[col])
    std_err = sem(df[col])
    h = std_err * t.ppf((1 + conf) / 2, n - 1)

    start = m - h
    end = m + h
    print(f"{conf} of data points fall between {round(start, sig_figs)} and {round(end, sig_figs)}")

In [0]:
import subprocess
import json
def add_mccabe_metrics(df, col):
    mccabe = []
    sloc = []
    comments = []
    ratio = []
    for i in range(len(df)):
        if i == 99:
            print("At file #100", end="")
        if ((i + 1) % 200) == 0:
            print(", file #" + str(i + 1), end="")
        with open(f"temp_file.{LANGUAGE}", "w", encoding='UTF-8') as fp:
            fp.write(train_df[col][i])
        out = subprocess.Popen(['metrics', '-q', '--format=json', f'temp_file.{LANGUAGE}'], 
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.STDOUT)
        stdout,stderr = out.communicate()
        metrics_data = json.loads(stdout)
        mccabe.append(metrics_data['files'][f'temp_file.{LANGUAGE}']['mccabe'])
        sloc.append(metrics_data['files'][f'temp_file.{LANGUAGE}']['sloc'])
        comments.append(metrics_data['files'][f'temp_file.{LANGUAGE}']['comments'])
        ratio.append(metrics_data['files'][f'temp_file.{LANGUAGE}']['ratio_comment_to_code'])
#         print(mccabe_data['files'][f'temp_file.{LANGUAGE}'].keys())
        os.remove(f"temp_file.{LANGUAGE}")
    df["complexity"] = mccabe
    df["code_lines"] = sloc
    df["comments"] = comments
    df["ratio_comment_to_code"] = ratio
    return df

# Analysis & Visualization

In [11]:
# Retrieve the data
corpus_df = simulate_getting_dataframes_from_mongo()
src_df = corpus_df.loc[corpus_df['data_type'] == 'src']
test_df = corpus_df.loc[corpus_df['data_type'] == 'test']

total_file_count = len(corpus_df)
src_files_count = len(src_df)
test_files_count = len(test_df)
print("Source Code Files:", src_files_count, "(" + str(round(src_files_count/total_file_count, 4)*100) + "%)")
print("Test Code Files:", test_files_count, "(" + str(round(test_files_count/total_file_count, 4)*100) + "%)")
print("Total files:", total_file_count)

Source Code Files: 14 (40.0%)
Test Code Files: 21 (60.0%)
Total files: 35


In [0]:
# Generate the SP Models
model = gen_sp_model(corpus_df, output='LIBest', model_name='_sp_bpe_modal', cols=['contents'])

In [13]:
corpus_df = add_length_col(corpus_df, 'contents', 'code_length')
stat_metrics(corpus_df, 'code_length')

~~Printing metrics for all data in column [code_length]~~
Min = 178
Max = 149212
Average = 29901.8857
Median = 28111
Standard Deviation = 26290.3033
0.95 of data points fall between 20739.0036 and 39064.7678


# Scratch Code (Testing)

In [0]:
print(round(1236721.8237468732, 3))